In [2]:
import numpy as np
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt
%matplotlib inline  
from sklearn.decomposition import NMF
from sklearn.decomposition import ProjectedGradientNMF
from numpy import array, matrix, linalg
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import TruncatedSVD
from scipy.linalg import svd

In [3]:
data = pd.read_csv('code/data/data.csv', delimiter = "\t")

In [4]:
data2 = pd.read_csv('code/data/questions', sep='\t')

In [5]:
data3 = pd.read_csv('code/data/questions_testing', sep='\t')

In [6]:
data3.head()

,Questions
0,I take time out for others.
1,I know that I am not a special person.
2,I take control of things.
3,I try to forgive and forget.
4,I keep in the background.


In [7]:
data2[data2.Description.str.startswith('I') == True].count()

Field          162
Format         162
Description    162
dtype: int64

In [18]:
data2[160:170]

,Field,Format,Description
160,P8,INTEGER,I am not easily annoyed rated on a five point ...
161,P9,INTEGER,I try to forgive and forget rated on a five po...
162,country,STRING,The country the user connected from (based on ...
163,"A1,""Anonymous Proxy""",NaN,NaN
164,"A2,""Satellite Provider""",NaN,NaN
165,"O1,""Other Country""",NaN,NaN
166,"AD,""Andorra""",NaN,NaN
167,"AE,""United Arab Emirates""",NaN,NaN
168,"AF,""Afghanistan""",NaN,NaN
169,"AG,""Antigua and Barbuda""",NaN,NaN


In [13]:
codebook_pre = data2['Description'].apply(lambda x: str(x).split(" rated")[0] + ".")[:162]


In [16]:
codebook = [i[1] for i in codebook_pre.iteritems()]


In [29]:
quiz = [i[1][0] for i in data3.iterrows()]

['I take time out for others.',
 'I know that I am not a special person.',
 'I take control of things.',
 'I try to forgive and forget.',
 'I keep in the background.',
 "I can't do without the company of others.",
 'I trust others.',
 'I am not easily frustrated.',
 'I cheer people up.',
 'I often feel uncomfortable around others.',
 'I seldom feel blue.',
 'I dislike myself.',
 'I take charge.',
 'I let others make the decisions.',
 'I believe in the importance of art.',
 'I like to get lost in thought.',
 'I wait for others to lead the way.',
 'I am willing to talk about myself.',
 'I find it difficult to approach others.',
 'I enjoy my privacy.',
 'I swim against the current.',
 'I feel guilty when I say "no."',
 'I am hard to get to know.',
 "I don't talk a lot.",
 'I believe in one true religion.',
 'I am not easily annoyed.',
 'I feel crushed by setbacks.',
 'I am afraid that I will do the wrong thing.',
 'I enjoy being part of a loud crowd.',
 'I weigh the pros against the cons.

In [33]:
for i in data3.iterrows():
    if i[1][0] not in codebook:
        print i[1][0]

I feel guilty when I say "no."
I have a good word for everyone.
I want everything to be "just right."


In [34]:
for i in codebook:
    if i not in quiz:
        print i

I feel guilty when I say 'no'.
I want everything to be 'just right'.


In [36]:
data2.head()

,Field,Format,Description
0,A1,INTEGER,I know how to comfort others rated on a five p...
1,A2,INTEGER,I enjoy bringing people together rated on a fi...
2,A3,INTEGER,I feel others' emotions rated on a five point ...
3,A4,INTEGER,I take an interest in other people's lives rat...
4,A5,INTEGER,I cheer people up rated on a five point scale ...


In [39]:
data2.shape

(422, 3)

In [40]:
data2[160:170]

,Field,Format,Description
160,P8,INTEGER,I am not easily annoyed rated on a five point ...
161,P9,INTEGER,I try to forgive and forget rated on a five po...
162,country,STRING,The country the user connected from (based on ...
163,"A1,""Anonymous Proxy""",NaN,NaN
164,"A2,""Satellite Provider""",NaN,NaN
165,"O1,""Other Country""",NaN,NaN
166,"AD,""Andorra""",NaN,NaN
167,"AE,""United Arab Emirates""",NaN,NaN
168,"AF,""Afghanistan""",NaN,NaN
169,"AG,""Antigua and Barbuda""",NaN,NaN


In [56]:
first = data2[0:162]
last = data2[162:]
new_row = pd.DataFrame([['P10','INTEGER', 'I have a good word for everyone.']], columns=['Field', 'Format', 'Description'])
first = first.append(new_row, ignore_index=True)
first = first.append(last, ignore_index=True)


,Field,Format,Description
160,P8,INTEGER,I am not easily annoyed rated on a five point ...
161,P9,INTEGER,I try to forgive and forget rated on a five po...
162,P10,INTEGER,I have a good word for everyone.
163,country,STRING,The country the user connected from (based on ...
164,"A1,""Anonymous Proxy""",NaN,NaN
165,"A2,""Satellite Provider""",NaN,NaN
166,"O1,""Other Country""",NaN,NaN
167,"AD,""Andorra""",NaN,NaN
168,"AE,""United Arab Emirates""",NaN,NaN
169,"AF,""Afghanistan""",NaN,NaN


SyntaxError: EOL while scanning string literal (<ipython-input-66-e13c10607bc7>, line 1)

In [65]:
data2['Description'].head()

0    I know how to comfort others rated on a five p...
1    I enjoy bringing people together rated on a fi...
2    I feel others' emotions rated on a five point ...
3    I take an interest in other people's lives rat...
4    I cheer people up rated on a five point scale ...
Name: Description, dtype: object

In [124]:
data.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,...,P7,P8,P9,P10,age,gender,accuracy,country,source,elapsed
0,1,4,2,3,3,2,3,4,4,3,...,1,2,5,2,17,1,92,US,6,914
1,4,3,4,3,4,4,4,4,2,2,...,3,2,2,2,37,1,100,US,1,891
2,3,4,4,4,4,4,4,3,2,2,...,2,3,3,3,31,1,80,US,6,903
3,4,5,4,4,4,3,3,2,2,2,...,2,3,4,4,32,1,93,US,1,806
4,4,0,4,4,4,3,5,1,2,4,...,4,2,3,2,46,2,87,NZ,1,1826


In [1]:
from incf.countryutils import transformations


In [5]:
data.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,...,P7,P8,P9,P10,age,gender,accuracy,country,source,elapsed
0,1,4,2,3,3,2,3,4,4,3,...,1,2,5,2,17,1,92,US,6,914
1,4,3,4,3,4,4,4,4,2,2,...,3,2,2,2,37,1,100,US,1,891
2,3,4,4,4,4,4,4,3,2,2,...,2,3,3,3,31,1,80,US,6,903
3,4,5,4,4,4,3,3,2,2,2,...,2,3,4,4,32,1,93,US,1,806
4,4,0,4,4,4,3,5,1,2,4,...,4,2,3,2,46,2,87,NZ,1,1826


In [14]:
test = transformations.cn_to_ctn('NZ')


KeyError: 'NZ'

In [10]:
test


'Europe'

In [16]:
transformations.cca_to_ctn('US')

'North America'

In [47]:
def get_continent(single_row):

    #country abreiviation
    try: 
        continent = transformations.cca_to_ctn(single_row)
    except:
        if single_row == "EU":
            continent = "Europe"
        elif single_row == "NaN" or single_row == "A2" or single_row == "A1" or single_row == "FX" or single_row == 'BQ':
            continent = "none"
        elif single_row == "AP":
            continent = "Asia"    
        else:
            continent = "none"
    return continent


In [48]:
data['Continent'] = data['country'].apply(lambda x: get_continent(x))

In [49]:
data[data['Continent'] == 'none']

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,...,P8,P9,P10,age,gender,accuracy,country,source,elapsed,Continent
3826,3,2,4,4,3,4,2,3,2,2,...,2,4,3,41,2,85,BQ,6,680,none
4284,2,4,4,4,4,4,4,2,2,1,...,4,4,3,26,1,70,A1,2,1635,none
5049,2,2,4,2,2,4,1,4,4,4,...,1,2,2,28,1,60,A2,1,1371,none
5135,4,4,4,4,5,5,3,2,2,1,...,4,4,4,31,2,100,A1,6,423,none
7307,4,5,0,4,5,1,4,2,1,3,...,4,5,4,18,2,95,A1,6,1004,none
8080,4,4,5,2,4,4,2,4,2,2,...,2,5,4,18,2,100,A1,6,1097,none
8406,4,4,4,4,4,3,4,2,2,2,...,2,4,4,16,0,75,A1,1,1339,none
16600,3,4,4,2,4,2,2,2,4,4,...,4,4,3,26,2,100,NaN,1,904,none
18248,5,1,5,1,1,1,5,5,5,2,...,1,5,1,53,1,100,NaN,1,743,none
19884,2,4,2,4,5,2,4,2,2,2,...,4,4,3,40,1,85,NaN,6,611,none


In [27]:
data['Continent'].unique

<bound method Series.unique of 0        North America
1        North America
2        North America
3        North America
4              Oceania
5               Europe
6        North America
7        North America
8        North America
9        North America
10       North America
11       North America
12       North America
13                Asia
14                Asia
15       North America
16              Europe
17                Asia
18       North America
19       North America
20       North America
21       North America
22       North America
23              Europe
24       North America
25       North America
26       North America
27       North America
28       North America
29       North America
             ...      
49129    North America
49130    North America
49131    North America
49132          Oceania
49133    North America
49134    North America
49135          Oceania
49136    North America
49137             Asia
49138    North America
49139    North America
491